# Halieutique

<!-- SUMMARY: Données du groupe de travail CIEM WGACEGG: densités acoustiques (NASC) d'anchois et de sardine par unité d'échantillonnage de 1 mille nautique de long, observées au printemps 2018 et 2021. -->

<!-- CATEGORY: Case_Studies -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import os
import geopandas as gpd
from shapely.geometry import Polygon

Le jeu de données du groupe de travail CIEM WGACEGG: densités acoustiques (NASC) d'anchois et de sardine par unité d'échantillonnage de 1 mille nautique de long, observées au printemps 2018 et 2021.

Description des colonnes : 
- survey : campagne
- year : année
- time : date/heure enregistrement donnée
- x : longitude en ° décimal WGS84
- y : latitude en ° décimal WGS84
- NASC : densité acoustique en m2.MN-2
- sp : espèce anchois = "ENGR-ENC"
- sardine ="SARD-PIL"

Pour plus d'infos sur l'origine de ces données : Doray, M., Van Der Kooij, J., Boyra, G. (Eds.), 2021. ICES Survey Protocols - Manual for acoustic surveys coordinated under the ICES Working Group on Acoustic and Egg Surveys for Small Pelagic Fish (WGACEGG). https://doi.org/10.17895/ICES.PUB.7462.

Lecture du fichier de données

In [ ]:
file_csv = gdoc.loadData("halieutic", "AC-SPRING-IBBB-NASC_ANE-PIL_2018-2021.csv")
rawDat = pd.read_csv(file_csv,sep=";")
rawDat.head()

Creation de la base de données

In [ ]:
dbDat = gl.Db()
for field in ["year","x","y","NASC"] :
    dbDat[field] = rawDat[field]

In [ ]:
dbDat.setLocators(["x","y"],gl.ELoc.X)
dbDat.setLocator("NASC",gl.ELoc.Z)

Conversion des coordonnées

In [ ]:
crsFrom="EPSG:4326"
crsTo="EPSG:2154"
a = gpd.points_from_xy(dbDat["x"], dbDat["y"], crs=crsFrom)
a = a.to_crs(crsTo)
dbDat["x"] = a.x
dbDat["y"] = a.y

Lecture du Polygone

In [ ]:
file_csv = gdoc.loadData("halieutic", "WGACEGGspringPolygon.csv")
csv = gl.CSVformat(True, 0, ";", ".")
poly = gl.Polygons.createFromCSV(file_csv, csv)

Conversion des coordonnées

In [ ]:
crsFrom="EPSG:4326"
crsTo="EPSG:2154"
a = gpd.points_from_xy(poly.getX(0), poly.getY(0), crs=crsFrom)
a = a.to_crs(crsTo)
poly.setX(0,a.x)
poly.setY(0,a.y)

Lectures des traits de côte.

In [ ]:
# Suppression des warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# Frontières des pays du monde dans un geopandaframe
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # Warning
# Voir https://github.com/geopandas/geopandas/issues/2751
# Alternative:
#import geodatasets as gda
#world = gpd.read_file(gda.get_path('naturalearth.land'))
# Focus sur le Portugal
minx=(-10,36)
miny=(-1,36)
maxx=(-1,49)
maxy=(-10,49)
polyGpd = Polygon([minx, miny, maxx, maxy])
world['geometry'] = world.geometry.intersection(polyGpd) # Warning

Conversion des coordonnées

In [ ]:
crsTo="EPSG:2154"
world = world.to_crs(crsTo)

Affichage des traits de côtes où l'on ajoute les points de données et le polygon utilisant la même CRS.

In [ ]:
ax = world.plot(figsize=[10,10])
ax = ax.gstpoint(dbDat,size=1)
ax = ax.polygon(poly)